In [2]:
import torch
print(torch.__version__)

2.3.0+cu121


In [3]:
!pip install transformers
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to c:\users\azmain\appdata\local\temp\pip-req-build-x9b68lae
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\Azmain\AppData\Local\Temp\pip-req-build-x9b68lae'


In [4]:
import clip
import os
from torch import nn
import numpy as np
import torch
import torch.nn.functional as nnf
import sys
from typing import Tuple, List, Union, Optional
from transformers import AutoTokenizer, GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm
import skimage.io as io
import PIL.Image
from IPython.display import Image 

D = torch.device
CPU = torch.device('cpu')

def get_device(device_id: int) -> D:
    if not torch.cuda.is_available():
        return CPU
    device_id = min(torch.cuda.device_count() - 1, device_id)
    return torch.device(f'cuda:{device_id}')

CUDA = get_device

current_directory = os.getcwd()
print(current_directory)
os.makedirs(current_directory, exist_ok=True)
model_path = os.path.join(current_directory, 'blekha_prefix-005.pt')
print(model_path)

f:\Anaconda\Jupyter Notebooks\CLIP_prefix_caption_Bangla_Dataset\FinalModels\RN50x4_shahidul034_bs8_blekha
f:\Anaconda\Jupyter Notebooks\CLIP_prefix_caption_Bangla_Dataset\FinalModels\RN50x4_shahidul034_bs8_blekha\coco_prefix-005.pt


In [5]:
class MLP(nn.Module):
    def forward(self, x: T) -> T:
        return self.model(x)

    def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) -1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            if i < len(sizes) - 2:
                layers.append(act())
        self.model = nn.Sequential(*layers)


class ClipCaptionModel(nn.Module):
    def get_dummy_token(self, batch_size: int, device: D) -> T:
        return torch.zeros(batch_size, self.prefix_length, dtype=torch.int64, device=device)

    def forward(self, tokens: T, prefix: T, mask: Optional[T] = None, labels: Optional[T] = None):
        embedding_text = self.gpt.transformer.wte(tokens)
        prefix_projections = self.clip_project(prefix).view(-1, self.prefix_length, self.gpt_embedding_size)
        #print(embedding_text.size()) #torch.Size([5, 67, 768])
        #print(prefix_projections.size()) #torch.Size([5, 1, 768])
        embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)
        if labels is not None:
            dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
            labels = torch.cat((dummy_token, tokens), dim=1)
        out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=mask)
        return out

    def __init__(self, prefix_length: int, prefix_size: int = 640):  ## 512 Vit || 640 Resnet
        super(ClipCaptionModel, self).__init__()
        self.prefix_length = prefix_length
        self.gpt = GPT2LMHeadModel.from_pretrained("shahidul034/text_generation_bangla_model")
        self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]
        if prefix_length > 10:  # not enough memory
            self.clip_project = nn.Linear(prefix_size, self.gpt_embedding_size * prefix_length)
        else:
            self.clip_project = MLP((prefix_size, (self.gpt_embedding_size * prefix_length) // 2, self.gpt_embedding_size * prefix_length))

In [6]:
#entry_length(default:67, Final:200)
#beam_size:(default:5, test:10)

def generate_beam(model, tokenizer, beam_size: int = 5, prompt=None, embed=None,
                  entry_length=120, temperature=1., stop_token: str = '<end>'):

    model.eval()

    tokens = None
    scores = None
    device = next(model.parameters()).device
    seq_lengths = torch.ones(beam_size, device=device)
    is_stopped = torch.zeros(beam_size, device=device, dtype=torch.bool)
    
    with torch.no_grad():
        if embed is not None:
            generated = embed
        else:
            if tokens is None:
                tokens = torch.tensor(tokenizer.encode(prompt))
                tokens = tokens.unsqueeze(0).to(device)
                generated = model.gpt.transformer.wte(tokens)
        
        for i in range(entry_length):
            outputs = model.gpt(inputs_embeds=generated)
            logits = outputs.logits
            logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
            logits = logits.softmax(-1).log()
            
            if scores is None:
                scores, next_tokens = logits.topk(beam_size, -1)
                generated = generated.expand(beam_size, *generated.shape[1:])
                next_tokens, scores = next_tokens.permute(1, 0), scores.squeeze(0)
                if tokens is None:
                    tokens = next_tokens
                else:
                    tokens = tokens.expand(beam_size, *tokens.shape[1:])
                    tokens = torch.cat((tokens, next_tokens), dim=1)
            else:
                logits[is_stopped] = -float(np.inf)
                logits[is_stopped, 0] = 0
                scores_sum = scores[:, None] + logits
                seq_lengths[~is_stopped] += 1
                scores_sum_average = scores_sum / seq_lengths[:, None]
                scores_sum_average, next_tokens = scores_sum_average.view(-1).topk(beam_size, -1)
                next_tokens_source = next_tokens // scores_sum.shape[1]
                seq_lengths = seq_lengths[next_tokens_source]
                next_tokens = next_tokens % scores_sum.shape[1]
                next_tokens = next_tokens.unsqueeze(1)
                tokens = tokens[next_tokens_source]
                tokens = torch.cat((tokens, next_tokens), dim=1)
                generated = generated[next_tokens_source]
                scores = scores_sum_average * seq_lengths
                is_stopped = is_stopped[next_tokens_source]
            
            next_token_embed = model.gpt.transformer.wte(next_tokens.squeeze()).view(generated.shape[0], 1, -1)
            generated = torch.cat((generated, next_token_embed), dim=1)
            
            # Stop if '।' is encountered
            decoded_sequence = tokenizer.decode(tokens[0].cpu().numpy())
            is_stopped = is_stopped + torch.tensor([stop_token in decoded_sequence for _ in range(beam_size)], device=device)
            
            if is_stopped.all():
                break
    
    scores = scores / seq_lengths
    output_list = tokens.cpu().numpy()
    output_texts = [tokenizer.decode(output[:int(length)]) for output, length in zip(output_list, seq_lengths)]
    order = scores.argsort(descending=True)
    output_texts = [output_texts[i] for i in order]
    return output_texts

In [8]:
is_gpu = True 
device = CUDA(0) if is_gpu else "cpu"
clip_model, preprocess = clip.load("RN50x4", device=device, jit=False)
tokenizer = GPT2Tokenizer.from_pretrained("shahidul034/text_generation_bangla_model")

In [9]:
prefix_length = 10

model = ClipCaptionModel(prefix_length)

model.load_state_dict(torch.load(model_path, map_location=CPU), strict=False) 

model = model.eval() 
device = CUDA(0) if is_gpu else "cpu"
model = model.to(device)

In [ ]:
print(current_directory)
eval_results_path = os.path.join(current_directory, "eval_results.txt")
print(eval_results_path)

f:\Anaconda\Jupyter Notebooks\CLIP_prefix_caption_Bangla_Dataset\FinalModels\RN50x4_shahidul034_bs8_blekha
f:\Anaconda\Jupyter Notebooks\CLIP_prefix_caption_Bangla_Dataset\FinalModels\RN50x4_shahidul034_bs8_blekha\eval_results.txt


In [ ]:
import os
import skimage.io as io
import PIL.Image
from tqdm import tqdm
import itertools

lst=[]
with open(eval_results_path, 'w', encoding='utf8') as newfile:
    f=open(r"F:\Anaconda\Jupyter Notebooks\CLIP_prefix_caption_Bangla_Dataset\data\blekha\annotations\test_captions_ids.txt", 'r', encoding='utf8')

    for i in tqdm(f, desc="Generating Captions"):
        i=i.replace('\n','')
        path=os.path.join(r"F:\Anaconda\Jupyter Notebooks\CLIP_prefix_caption_Bangla_Dataset\data\blekha\test", i)
        
        # Read and preprocess the image
        image = io.imread(path)
        pil_image = PIL.Image.fromarray(image)
#         display(pil_image)
        image = preprocess(pil_image).unsqueeze(0).to(device)

        # Encode the image using CLIP model
        with torch.no_grad():
            prefix = clip_model.encode_image(image).to(device, dtype=torch.float32)
            prefix_embed = model.clip_project(prefix).reshape(1, prefix_length, -1)

        # Generate captions
        generated_text_prefix = generate_beam(model, tokenizer, embed=prefix_embed)[0]
        
        print(i," ",generated_text_prefix)
        caption = generated_text_prefix.split()
        newfile.write((i+str(caption))+'\n')

Generating Captions: 1it [00:00,  4.83it/s]

8875.png   <start> তিনটি শিশু আছে। <end>


Generating Captions: 2it [00:00,  3.45it/s]

2215.png   <start> একজন নারী মাথায় গামছা বেধে আছে। <end>


Generating Captions: 4it [00:01,  2.81it/s]

...

5157.png   <start> সবুজের ধানের ক্ষেত ও নদী দেখা যাচ্ছে। <end>


Generating Captions: 914it [04:32,  3.49it/s]

8231.png   <start> একজন মানুষ হেঁটে যাচ্ছে। <end>


Generating Captions: 916it [04:33,  3.35it/s]

5311.png   <start> কয়েকজন বন্ধু একসাথে পাহারের ঝর্ণার পানিতে গোসল করছে। <end>
9144.png   <start> একটি কুকুর আছে। <end>


In [ ]:
with open(eval_results_path, 'r', encoding='utf-8') as file:
    content = file.read()

content = content.replace("'<start>', " , '')
content = content.replace(", '<end>'", '')

with open(eval_results_path, 'w', encoding='utf-8') as file:
    file.write(content)

In [ ]:
!pip install nltk

In [ ]:
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

f = open(eval_results_path, 'r', encoding = 'utf8')
cans = []
for line in f:
    
    index = line.find('.png')
    line = line[index+4:].strip()
    line = line.replace("'",'')
    line = line.replace(",",'')
    line = line.replace("]",'')
    line = line.replace("[",'')
    cans.append([line])
# for i in cans:
#     print([i])

f1 = open(r"F:\Anaconda\Jupyter Notebooks\CLIP_prefix_caption_Bangla_Dataset\data\blekha\annotations\test_captions.txt",'r',encoding = 'utf8')
refs = []

c = 0
for line in f1:
    c+=1
    if(c%2 != 0):
        refs.append(line[10:].strip())
        #print("1", line[10:].strip())
    else:
        #print("2", line[10:].strip())
        refs.append(line[10:].strip())
        c  = 0
        #print('\n')
    
final_refs = []
for i in range(0,len(refs)-1,2):
    final_refs.append([refs[i],refs[i+1]])
# for i in final_refs:
#     print([i],len([i]))

b1,b2,b3,b4 = [],[],[],[]
# for i in range(1000):
for i in range(len(final_refs)):
    # Apply smoothing function to handle zero counts
    smoothing = SmoothingFunction().method2    

    # print(final_refs[i])
    score1 = corpus_bleu([final_refs[i]],cans[i], weights=(1, 0, 0, 0), smoothing_function=smoothing)
    score2 = corpus_bleu([final_refs[i]],cans[i], weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing)
    score3 = corpus_bleu([final_refs[i]],cans[i], weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothing)
    score4 = corpus_bleu([final_refs[i]],cans[i], weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing)
    #print(score1,score2,score3,score4)
    b1.append(score1)
    b2.append(score2)
    b3.append(score3)
    b4.append(score4)
print('BLEU 1: ',sum(b1)/len(b1))
print('BLEU 2: ',sum(b2)/len(b2))
print('BLEU 3: ',sum(b3)/len(b3))
print('BLEU 4: ',sum(b4)/len(b4))

print('\ntaking top 500 bleu:\n')
b1.sort(reverse=True)
b2.sort(reverse=True)
b3.sort(reverse=True)
b4.sort(reverse=True)
top_k = 500
print('BLEU 1: ',sum(b1[:top_k])/top_k)
print('BLEU 2: ',sum(b2[:top_k])/top_k)
print('BLEU 3: ',sum(b3[:top_k])/top_k)
print('BLEU 4: ',sum(b4[:top_k])/top_k)

BLEU 1:  0.6742612129398216
BLEU 2:  0.61471824541877
BLEU 3:  0.5660176230045902
BLEU 4:  0.5222990606994389

taking top 500 bleu:

BLEU 1:  0.864701116045749
BLEU 2:  0.8009602216074249
BLEU 3:  0.7549800599530906
BLEU 4:  0.7142094717586239


### 10 epoch score

BLEU 1:  0.6887589489620958

BLEU 2:  0.6249751048654124

BLEU 3:  0.5747700033048548

BLEU 4:  0.5297908680629894

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.translate.meteor_score import meteor_score

f = open(eval_results_path, 'r', encoding = 'utf8')
cans = []
for line in f:
    
    index = line.find('.png')
    line = line[index+4:].strip()
    line = line.replace("'",'')
    line = line.replace(",",'')
    line = line.replace("]",'')
    line = line.replace("[",'')
    #line = line.split()
    cans.append(line)
#for i in cans:
#    print([i])

#print(len(cans))

f1 = open(r"F:\Anaconda\Jupyter Notebooks\CLIP_prefix_caption_Bangla_Dataset\data\blekha\annotations\test_captions.txt",'r',encoding = 'utf8')
refs = []
test_ids = []

for line in f1:
    v = line.split()[0]
    if v not in test_ids:
        test_ids.append(v)
#print(len(test_ids))   
f1 = open(r"F:\Anaconda\Jupyter Notebooks\CLIP_prefix_caption_Bangla_Dataset\data\blekha\annotations\test_captions.txt",'r',encoding = 'utf8')
new = []
temp = []
final_refs = []
for line in f1:
    x = line.split()[0]
    new.append(x)
    if x in test_ids:
        line  = line.replace(x,'')
        line = line.strip()
        temp.append(line)
    if(len(temp)==2):                   #==2 for banglalekha dataset
        final_refs.append(temp)
        #print(temp)
        temp = []
    
print(len(final_refs),len(new))

m_score = []
for i in range(len(final_refs)):
    reference_set = final_refs[i]
    generated_set = cans[i]
    
    # Tokenize hypothesis and reference sentences
    hypothesis_tokens = nltk.word_tokenize(generated_set)
    reference_tokens = [nltk.word_tokenize(ref) for ref in reference_set]
    
    meteor = round(meteor_score(reference_tokens, hypothesis_tokens), 5)
    if(i<10):
        print(reference_set, generated_set, meteor)
    # meteor = round(meteor_score(final_refs[i],cans[i]),5)
    # print(final_refs[i],cans[i],meteor)
    
    m_score.append(meteor)

print('\nMeteor:',(sum(m_score)/len(m_score)))
print('\ntaking top 500:\n')
m_score.sort(reverse=True)

top_k = 500
print('Meteor: ',sum(m_score[:top_k])/top_k)

916 1832
['কয়েকজন ছেলে-মেয়ে লঞ্চের দড়ি ধরে ঝুলছে আর কয়েকজন কাঠের সিঁড়ি বেয়ে উপরে উঠছে।', 'কয়েকজন শিশু আছে।'] তিনটি শিশু আছে। 0.625
['রঙে মাখা জামা পরা একটা মেয়ে এক হাতে রঙের ব্যাগ আর আরেক হাতে রং নিয়ে ফু দিয়ে রং উড়াচ্ছে।', 'একজন নারী রঙ মেখে আছে।'] একজন নারী মাথায় গামছা বেধে আছে। 0.50109
['লাল জামা পরিহিত একজন নারী কালো জামা পরিহিত একজন নারীকে জড়িয়ে ধরেছে। একজন পুরুষ একটি শিশুকে কোলে নিয়ে আছে। পিছনে কয়েকজন মানুষ দাড়িয়ে আছে।', 'দুইজন মেয়ে একসাথে গলা জড়িয়ে আর একটি লোক একটি বাচ্চাকে কোলে নিয়ে ছবি তুলছে।'] একজন পুরুষ একটি শিশু কোলে নিয়ে আছে। আশেপাশে অনেকগুলো মানুষ আছে। 0.3304
['দুইটি শিশু আছে।', 'দুইজন ছোট ছেলে দাড়িয়ে আছেন যার একজন কাপড় পরে আছে আরেকজন হাফ প্যান্ট পরে আছেন।'] দুইটি শিশু আছে। 0.98148
['একটি শিশু দাড়িয়ে আছে।', 'একটি  ঘোড়ার গাড়ির সাথে দাড়িয়ে আছে একটি ছোট ছেলে।'] একজন বালক সাইকেল চালাচ্ছে। 0.0
['একজন নারী কলশী নিয়ে আছে।', 'একজন শারি পরা মহিলা কাখে কলসি নিয়ে দাড়িয়ে আছে।'] একজন নারী দাড়িয়ে আছে। 0.52154
['একটি শিশু তাকিয়ে আছে।', 'একটি বাচ্চা মেয়ে দরজা ধরে দারিয়ে আছে।'] একটি মেয়ে শ

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Azmain\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Meteor: 0.3802769104803497

taking top 500:

Meteor:  0.5915762200000005


Bornon paper best result: (Taken from their paper)
Split : 78-11-11
BLEU 1:  0.665
BLEU 2:  0.556
BLEU 3:  0.476
BLEU 4:  0.408

Our bs32 result:
Split : 81-9-10
BLEU 1:  0.622
BLEU 2:  0.554
BLEU 3:  0.502
BLEU 4:  0.451